# Advanced guide for data modeling based cogshop runs

**This tutorial covers more advanced techniques than quickstart and will not go into details covered there, such as setting up the Powerops Client**


## Preparation


NOTE: The "recommended" way of running CogShop is as follows:
1. Static model data and mappings for time-dependent data like time series represented by `ShopModel`
2. Any variations on the base configuration (like different price scenarios) represented by `ShopScenario` instances 
3. Generate a complete `ShopCase` with time series datapoints based on a `ShopScenario` instance plus `startTime` and `endTime` through the shop_trigger Cognite function.

This tutorial covers a different set-up: Triggering (Cog)Shop for a pre-generated, complete Shop "case" (as a yaml file or separate files).


In [1]:
# You can control which configuration files are used.
# In this case, the configuration files are located two levels above, in the root of the repository.
from dotenv import load_dotenv
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient


# Adjust the path to the config files if needed
load_dotenv(dotenv_path="../../.env")
powerops = PowerOpsClient.from_config("../../power_ops_config.yaml")

print(__version__)  # Print the version of the package

0.103.0


In [2]:
# Creating some helper functions

from pathlib import Path


def upload_file(local_file_name: str, external_id: str) -> str:
    """Upload a file to CDF and return the external id of the file"""
    file_path = (Path("example_case_files") / local_file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id=external_id,
        name=local_file_name,
        data_set_id=powerops.datasets.write_dataset_id,
        mime_type="application/yaml",
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id

## Example 1: Use multiple files for shop files
Very similar to the example in quickstart, but using multiple files for `ShopCase`.

1. Upload the files that will be loaded into shop, keeping their external ids
2. Set up a `ShopCase`, using the SDK
3. Write the `ShopCase` instance to CDF, and verify that it was created using the SDK
4. Trigger a shop execution of that `ShopCase`
5. List the `ShopResult` referencing the `ShopCase` 


### Step 1: Upload a files 

In [3]:
# NB! The case in this example is minimal and just for demo purposes.
example_case_fbu_1 = upload_file(
    local_file_name="b_example_fornebu_without_commands.yaml",
    external_id="example_case_fornebu",
)
example_case_fbu_2 = upload_file(
    local_file_name="b_example_commands.yaml", external_id="example_commands"
)

print(
    f" File reference 1: '{example_case_fbu_1}', File reference 2: '{example_case_fbu_2}'"
)

 File reference 1: 'example_case_fornebu', File reference 2: 'example_commands'


### Step 2: Preparing a ShopCase

The main difference here is that `shop_file_list` will contain more than one tuple.

Unlike the quickstart, in this example we will let `prepare_shop_case` generate all the external IDs.

We will also let the file reference double as the name of the file. 


In [4]:
import datetime


#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]


shop_case_1 = powerops.cogshop.prepare_shop_case(
    shop_file_list=shop_file_list,
    shop_version="15.6.1.0",
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    model_name="my_model",
    scenario_name="my_scenario",
)
shop_case_1

,value
space,power_ops_instances
external_id,shopcase:452ba4f84d534ac38ad78356b3d10e93
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [5]:
# We can also look at just the shop files
for file in shop_case_1.shop_files:
    print(
        f"File name: '{file.name}', File reference: '{file.file_reference}' \t[,data modeling external id: '{file.external_id}']"
    )

File name: 'example_case_fornebu', File reference: 'example_case_fornebu' 	[,data modeling external id: 'shopfile:245b7a660c224c8da69d60f6280bad8c']
File name: 'example_commands', File reference: 'example_commands' 	[,data modeling external id: 'shopfile:353ca5327e8d409988132e7c0f26639e']


### Step 3: Upload the ShopCase

In [6]:
powerops.v1.upsert(shop_case_1)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopcase:452ba4f84d534ac38ad78356b3d10e93', version=1) at 0x7f001f7e4510>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:245b7a660c224c8da69d60f6280bad8c', version=1) at 0x7f001f7d1a50>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:353ca5327e8d409988132e7c0f26639e', version=1) at 0x7f0020177d10>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:c959c876201f435ca770bcf1a28ad47a', version=1) at 0x7f0020176e90>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:f337d96c4ad943c1a4232e2423bbfc6e', version=1) at 0x7f0020177590>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='shopcase:452ba4f84d534ac38ad78356b3d10e93:shopfile:245b7a660c224c8da69d60f6280bad8c', version=1) at 0x7f001f80cd90>, <EdgeApplyResult(space='power_ops_instances', external_id='shopcase:452ba4f84d534ac38ad78356b3d10e93:shopfile:353

In [7]:
from cognite.powerops.client._generated.v1.data_classes import ShopCase

# NB! This step is not necessary, if the upsert was successful we know the case is in CDF
retrieved_shop_case: ShopCase = powerops.cogshop.retrieve_shop_case(
    shop_case_1.external_id
)
retrieved_shop_case

,value
space,power_ops_instances
external_id,shopcase:452ba4f84d534ac38ad78356b3d10e93
data_record,"{'version': 1, 'last_updated_time': 2024-11-14..."
node_type,None
scenario,shopscenario:c959c876201f435ca770bcf1a28ad47a
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[shopfile:245b7a660c224c8da69d60f6280bad8c, sh..."


*It is known that the `retrive` endpoint can be unstable*. 
We have a graphQL fallback for this case.  This is a less efficient way to retrieve the shop case, but it is useful in case the `retrive` fails

See details in the docstring of `retrieve_shop_case_graphql`

In [ ]:
graph_ql_shop_case = powerops.cogshop.retrieve_shop_case_graphql(
    shop_case_1.external_id
)
graph_ql_shop_case

,value
space,power_ops_instances
external_id,shopcase:452ba4f84d534ac38ad78356b3d10e93
data_record,"{'version': 0, 'last_updated_time': 2024-11-14..."
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 4: Trigger the case

We will here directly use the new `trigger_shop_case` method on the `powerops.cogshop` client

In [10]:
powerops.cogshop.trigger_shop_case(shop_case_1.external_id)

### Step 5: View the ShopResult generated 

It may take a while to run (Cog)Shop.

A new things to note:
* If the run is not completed then no results are returned. 
* If a result is returned, but several values are `None` (namely `post run`, `messages`, `cplex`), it can be assumed that the run failed.
* Technically it is possible to run `trigger_shop_case` multiple times for the same case. For this reason we suggest using lists to view `ShopResult`s. 
  

Explanations for (some) of the fields of the `ShopResult`:

* External id: The external id of the `ShopResult`
* Case: The external id of the `ShopCase` that the instance belongs to
* Objective value: The objective value of the Shop execution
* Pre run: A file reference to the pre run yaml file
* Post run: A file reference to the post run yaml file
* Messages: A file reference to the logs generated by Shop
* Cplex logs: A file reference to the cplex logs generated by Shop
* Data record: The data record of the instance, contains the keys `last_updated_time` and `created_time`. These can be used to differentiate results from the same `ShopCase`



#### 1. Using list

Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned

In [13]:
from cognite.powerops.client._generated.v1.data_classes import ShopResultList

result_list_1: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_1.external_id
)  # This can easily be converted to a pandas DataFrame

print(
    f"There is/are {len(result_list_1)} result(s) for the case with external id '{shop_case_1.external_id}'"
)

There is/are 1 result(s) for the case with external id 'shopcase:452ba4f84d534ac38ad78356b3d10e93'


#### 2. Graph QL list 

As with the graphQL above, this is a less efficient way to retrieve shop results, but it can useful in case `list` does not preform well.

See detailed in the docstring of `list_shop_results_graphql`

In [ ]:
from cognite.powerops.client._generated.v1.data_classes import ShopResult

result_list_ql_1: list[ShopResult] = powerops.cogshop.list_shop_results_graphql(
    case_external_id=shop_case_1.external_id, limit=1
)
result_list_ql_1[0]

,value
space,power_ops_instances
external_id,shop_result__d7329327-5c92-4c92-bafa-f94ac1aca550
data_record,"{'version': 0, 'last_updated_time': 2024-11-14..."
node_type,None
case,"{'space': 'power_ops_instances', 'external_id'..."
objective_value,None
pre_run,POWEROPS_SHOP_pre-run-d7329327-5c92-4c92-bafa-...
post_run,POWEROPS_SHOP_post-run-d7329327-5c92-4c92-bafa...
messages,POWEROPS_SHOP_shop-d7329327-5c92-4c92-bafa-f94...
cplex_logs,POWEROPS_SHOP_cplex-d7329327-5c92-4c92-bafa-f9...


#### 3. Retrieve using the external ID  

In [15]:
shop_result_1: ShopResult = powerops.cogshop.retrieve_shop_result(
    result_list_1[0].external_id
)
shop_result_1

,value
space,power_ops_instances
external_id,shop_result__d7329327-5c92-4c92-bafa-f94ac1aca550
data_record,"{'version': 1, 'last_updated_time': 2024-11-14..."
node_type,None
case,shopcase:452ba4f84d534ac38ad78356b3d10e93
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-d7329327-5c92-4c92-bafa-...
post_run,POWEROPS_SHOP_post-run-d7329327-5c92-4c92-bafa...
messages,POWEROPS_SHOP_shop-d7329327-5c92-4c92-bafa-f94...
cplex_logs,POWEROPS_SHOP_cplex-d7329327-5c92-4c92-bafa-f9...


In [ ]:
# Downloading one of the files to tmp directory and printing the first 10 lines
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp_dir:
    pre_run_file_path = str(Path(tmp_dir) / "log_file.log")
    powerops.cdf.files.download_to_path(
        external_id=shop_result_1.messages, path=pre_run_file_path
    )
    with open(pre_run_file_path, "r") as log_file:
        print("".join(log_file.readlines()[:10]))

INFORMATION: 1032
15.6.1.0 Cplex 20.1.0 Gurobi 7.5 OSI/CBC 2.9 2024-05-10

INFORMATION: 1047
Current time: Thu Nov 14 10:39:31 2024

DIAGNOSIS WARNING: 3250
Reservoir Dalbysvatn: Only max tactical cost or limit is given.
DIAGNOSIS WARNING: 3205
Reservoir Dalbysvatn: 0 errors 1 warnings.



### Step 4 (optional): Prepare to delete created resources

In [18]:
all_external_ids_1 = [
    shop_case_1.scenario.external_id,
    shop_case_1.scenario.model.external_id,
    shop_case_1.external_id,
    shop_case_1.shop_files[0].external_id,
    shop_case_1.shop_files[1].external_id,
    shop_result_1.external_id,
]

all_external_ids_1  # Record these for deletion

['shopscenario:c959c876201f435ca770bcf1a28ad47a',
 'shopmodel:f337d96c4ad943c1a4232e2423bbfc6e',
 'shopcase:452ba4f84d534ac38ad78356b3d10e93',
 'shopfile:245b7a660c224c8da69d60f6280bad8c',
 'shopfile:353ca5327e8d409988132e7c0f26639e',
 'shop_result__d7329327-5c92-4c92-bafa-f94ac1aca550']

## Example 2: Reusing a ShopScenario instance
As mentioned in the intro Quickstart, the recommend way to run CogShop is by having a static model and use scenarios to vary configurations. 

However, if we know that we want to run several `ShopCases` using the same configurations, we can reuse a `ShopScenario`/`ShopModel` instead of creating several identical instances.


1. Create a `ShopScenario` and `ShopModel` that set the desired configuration (here `shop_version`). We will create two for demo purposes using the data classes directly 
2. Write the `ShopScenario` to CDF. 
3. Set up a `ShopCase`, preparing shop file list with names and references as explained in Example 1. Showing two ways to reference `ShopScenario`
4. Write the cases to  to CDF. 
4. Trigger shop executions of the `ShopCase`a  and list the `ShopResult` as in previous examples 

As in the previous example, we will be letting the SDK generate all external id. Again they can be 

### Step 1: Create reusable ShopScenarios (with ShopScenarios)

`ShopScenario`s and `ShopModel`s created this way, will have several unused fields that are `None`. This is due to the there usage here being very different from their intended usage.   


See the Power Ops SDK or the Data Models in Fusion for more information of what the fields represent

In [27]:
# Not necessary, but we can list available files Shop Version
powerops.cogshop.list_shop_versions()

['SHOP-15.4.1.1-pyshop-python3.11-linux.zip',
 'SHOP-16.0.3-pyshop-python3.11-linux.zip',
 'SHOP-16.0.2-pyshop-python3.11-linux.zip',
 'SHOP-15.6.1.0-pyshop-python3.11-linux.zip']

In [38]:
from cognite.powerops.client._generated.v1.data_classes import ShopModelWrite, ShopScenarioWrite

# Using shop version 15.6.1.0
reusable_model_15_6_1_0 = ShopModelWrite(
    name="reusable_shop_model_15_6_1_0",
    shop_version="15.6.1.0",
    model_version="1.0",
    model=None, 
    penalty_limit=None, 
    cog_shop_version=None,
    base_attribute_mappings=None,
)

reusable_scenario_15_6_1_0 = ShopScenarioWrite(
        name="reusable_shop_scenario_15_6_1_0",
        model=reusable_model_15_6_1_0,
        commands=None,
        source=None,
        output_definition=None,
        attribute_mappings_override=None,
    )        

# Using shop version 16.0.3
reusable_model_16_0_3 = ShopModelWrite(
    name="reusable_shop_model_16_0_3",
    shop_version="16.0.3",
    model_version="1.0",
    model=None, 
    penalty_limit=None, 
    cog_shop_version=None,
    base_attribute_mappings=None,
)

reusable_scenario_16_0_3 = ShopScenarioWrite(
        name="reusable_shop_scenario_16_0_3",
        model=reusable_model_16_0_3,
        commands=None,
        source=None,
        output_definition=None,
        attribute_mappings_override=None,
    )

In [39]:
# Looking at the models
print(f"Shop Model 15.5.1.0 '{reusable_model_15_6_1_0.external_id}'")
print(f"Shop Model 16.0.3   '{reusable_model_16_0_3.external_id}'")

Shop Model 15.5.1.0 'shopmodel:d37fce4661d44014a23f292c1098b1a8'
Shop Model 16.0.3   'shopmodel:697585639fbc4c82b04b2f7b0cea500a'


In [40]:
# Looking at the scenarios
print(f"Shop Scenario 15.6.1.0: '{reusable_scenario_15_6_1_0.external_id}'")
print(f"Shop Scenario 16.0.3:   '{reusable_scenario_16_0_3.external_id}'")

Shop Scenario 15.6.1.0: 'shopscenario:4ed8eecfe7484e21b5baf1cc616f47a2'
Shop Scenario 16.0.3:   'shopscenario:eef230c5d6284153b88e29232b8ad5af'


### Step 2: Write the scenarios to CDF

Since the Scenarios are referencing `Write` instances of the `ShopModel`s, just upserting the scenarios is sufficient. 
Here we will only upsert 1 of the scenarios to demonstrate two different ways to prepare a case with an existing scenario.

In [49]:
powerops.v1.upsert(reusable_scenario_15_6_1_0)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopscenario:4ed8eecfe7484e21b5baf1cc616f47a2', version=1) at 0x7f001d4bf990>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:d37fce4661d44014a23f292c1098b1a8', version=1) at 0x7f001d4bfed0>], edges=[], time_series=[])

### Step 3: Create new Shop Cases, referencing the newly created scenarios

We will be reusing the files from the example above

In [50]:
#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]

In [51]:
shop_case_15_6_1_0 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    shop_scenario_reference=reusable_scenario_15_6_1_0.external_id,  # Note that we are using the external id of the scenario
)
shop_case_15_6_1_0

,value
space,power_ops_instances
external_id,shopcase:e1efa9a4c0584419a856c54d504f4fcd
data_record,{'existing_version': None}
node_type,None
scenario,shopscenario:4ed8eecfe7484e21b5baf1cc616f47a2
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [ ]:
# This will fail because the scenario does not exist in CDF yet (we did not upsert it)
will_fail_shop_case_16_0_3 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    shop_scenario_reference=reusable_scenario_16_0_3.external_id,  # Note that this is the external id of the scenario
)

ValueError: Invalid shop scenario reference: shopscenario:eef230c5d6284153b88e29232b8ad5af.

In [ ]:
# However, if we reference the Write object in the case creation it will be work. 
# This is because a chained series of Write obejcts (which `prepare_shop_case_with_existing_scenario` returns) can be upserted at the same time.
shop_case_16_0_3 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    shop_scenario_reference=reusable_scenario_16_0_3,  # Note that this is the Write object
)
shop_case_16_0_3

,value
space,power_ops_instances
external_id,shopcase:6454261bed644b159e01016064fe5790
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 4: Upserting the new cases. 

Notice that the resources are bundled together as a list when we write more than one resource 

In [58]:
powerops.v1.upsert([shop_case_15_6_1_0, shop_case_16_0_3])

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopcase:e1efa9a4c0584419a856c54d504f4fcd', version=1) at 0x7f001d570d90>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:8d5bff7e334e438dbd7e6a22d558225e', version=1) at 0x7f001f84f350>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:6c8bd87ba57841d6be9ac50a2c34add7', version=1) at 0x7f001f84eb90>, <NodeApplyResult(space='power_ops_instances', external_id='shopcase:6454261bed644b159e01016064fe5790', version=1) at 0x7f001d821210>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:0fefbc2bd6ef4105934e66a73e7b7b0c', version=1) at 0x7f001d4af8d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:3f786d18ac0b46da83f6081aa6e24772', version=1) at 0x7f001f84fa90>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:eef230c5d6284153b88e29232b8ad5af', version=1) at 0x7f001d604710>, <NodeApplyResult(space='power_

### Step 5: Trigger Shop Executions and inspect results

In [59]:
powerops.cogshop.trigger_shop_case(shop_case_15_6_1_0.external_id)
powerops.cogshop.trigger_shop_case(shop_case_16_0_3.external_id)


Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned. 
If a result is returned, but several values are `None` (namely `post run`, `messages`, `cplex`), it can be assumed that the run failed.


In [60]:
result_list_15_5_1_0: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_15_6_1_0.external_id
)
print(
    f"There is/are {len(result_list_15_5_1_0)} result(s) for the case '{shop_case_15_6_1_0.external_id}'"
)

There is/are 1 result(s) for the case 'shopcase:e1efa9a4c0584419a856c54d504f4fcd'


In [61]:
shop_result_15_6_1_0: ShopResult = powerops.cogshop.retrieve_shop_result(
    result_list_15_5_1_0[0].external_id
)
shop_result_15_6_1_0

,value
space,power_ops_instances
external_id,shop_result__7df1c42b-fc68-4895-b666-93d3cfb5a60b
data_record,"{'version': 1, 'last_updated_time': 2024-11-14..."
node_type,None
case,shopcase:e1efa9a4c0584419a856c54d504f4fcd
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-7df1c42b-fc68-4895-b666-...
post_run,POWEROPS_SHOP_post-run-7df1c42b-fc68-4895-b666...
messages,POWEROPS_SHOP_shop-7df1c42b-fc68-4895-b666-93d...
cplex_logs,POWEROPS_SHOP_cplex-7df1c42b-fc68-4895-b666-93...


In [ ]:
result_list_16_0_3: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_16_0_3.external_id, limit=1
)
print(
    f"There is/are {len(result_list_16_0_3)} result(s) for the case '{shop_case_16_0_3.external_id}'"
)

There is/are 1 result(s) for the case 'shopcase:6454261bed644b159e01016064fe5790'


In [63]:
shop_result_16_0_3: ShopResult = powerops.cogshop.retrieve_shop_result(
    result_list_16_0_3[0].external_id
)
shop_result_16_0_3

,value
space,power_ops_instances
external_id,shop_result__cf536392-48a0-4e1e-910c-9bcbe7f848f9
data_record,"{'version': 1, 'last_updated_time': 2024-11-14..."
node_type,None
case,shopcase:6454261bed644b159e01016064fe5790
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-cf536392-48a0-4e1e-910c-...
post_run,POWEROPS_SHOP_post-run-cf536392-48a0-4e1e-910c...
messages,POWEROPS_SHOP_shop-cf536392-48a0-4e1e-910c-9bc...
cplex_logs,POWEROPS_SHOP_cplex-cf536392-48a0-4e1e-910c-9b...


Looking at the `pre_run`s we can see that the cases/results generated were using different versions of SHOP, while the model is the the same one.

In [ ]:
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp_dir:
    pre_run_file_path = str(Path(tmp_dir) / "pre_run.yaml")
    powerops.cdf.files.download_to_path(
        external_id=shop_result_15_6_1_0.pre_run, path=pre_run_file_path
    )
    with open(pre_run_file_path, "r") as log_file:
        print("".join(log_file.readlines()[:15]))

print("==========================================\n")

with TemporaryDirectory() as tmp_dir:
    pre_run_file_path = str(Path(tmp_dir) / "pre_run.yaml")
    powerops.cdf.files.download_to_path(
        external_id=shop_result_16_0_3.pre_run, path=pre_run_file_path
    )
    with open(pre_run_file_path, "r") as log_file:
        print("".join(log_file.readlines()[:15]))

time:
  starttime: 2023-01-19 23:00:00
  endtime: 2023-01-29 23:00:00
  timeunit: minute
  timeresolution:
    2023-01-19 23:00:00: 60
    2023-01-22 23:00:00: 240
model:
  global_settings:
    global_settings:
      shop_version: 15.6.1.0 Cplex 20.1.0 Gurobi 7.5 OSI/CBC 2.9 2024-05-10
  creek_intake:
    Blakstad:
      net_head: 641
      max_inflow: 12


time:
  starttime: 2023-01-19 23:00:00
  endtime: 2023-01-29 23:00:00
  timeunit: minute
  timeresolution:
    2023-01-19 23:00:00: 60
    2023-01-22 23:00:00: 240
model:
  global_settings:
    global_settings:
      shop_version: 16.0.3 Cplex 20.1.0 Gurobi 7.5 OSI/CBC 2.9 2024-08-16
  creek_intake:
    Blakstad:
      net_head: 641
      max_inflow: 12



### Step 5 (optional): Prepare to delete created resources

In [80]:
all_external_ids_2 = [ 
    reusable_scenario_15_6_1_0.external_id,
    reusable_scenario_15_6_1_0.model.external_id,
    reusable_scenario_16_0_3.external_id,
    reusable_scenario_16_0_3.model.external_id,

    shop_case_15_6_1_0.external_id,
    shop_case_15_6_1_0.shop_files[0].external_id,
    shop_case_15_6_1_0.shop_files[1].external_id,
    
    shop_case_16_0_3.external_id,
    shop_case_16_0_3.shop_files[0].external_id,
    shop_case_16_0_3.shop_files[1].external_id,
    
    shop_result_15_6_1_0.external_id,
    shop_result_16_0_3.external_id,

]
all_external_ids_2 # Record these for deletion

['shopscenario:4ed8eecfe7484e21b5baf1cc616f47a2',
 'shopmodel:d37fce4661d44014a23f292c1098b1a8',
 'shopscenario:eef230c5d6284153b88e29232b8ad5af',
 'shopmodel:697585639fbc4c82b04b2f7b0cea500a',
 'shopcase:e1efa9a4c0584419a856c54d504f4fcd',
 'shopfile:8d5bff7e334e438dbd7e6a22d558225e',
 'shopfile:6c8bd87ba57841d6be9ac50a2c34add7',
 'shopcase:6454261bed644b159e01016064fe5790',
 'shopfile:0fefbc2bd6ef4105934e66a73e7b7b0c',
 'shopfile:3f786d18ac0b46da83f6081aa6e24772',
 'shop_result__7df1c42b-fc68-4895-b666-93d3cfb5a60b',
 'shop_result__cf536392-48a0-4e1e-910c-9bcbe7f848f9']

## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [81]:
all_external_ids = all_external_ids_1 + all_external_ids_2
all_external_ids

['shopscenario:c959c876201f435ca770bcf1a28ad47a',
 'shopmodel:f337d96c4ad943c1a4232e2423bbfc6e',
 'shopcase:452ba4f84d534ac38ad78356b3d10e93',
 'shopfile:245b7a660c224c8da69d60f6280bad8c',
 'shopfile:353ca5327e8d409988132e7c0f26639e',
 'shop_result__d7329327-5c92-4c92-bafa-f94ac1aca550',
 'shopscenario:4ed8eecfe7484e21b5baf1cc616f47a2',
 'shopmodel:d37fce4661d44014a23f292c1098b1a8',
 'shopscenario:eef230c5d6284153b88e29232b8ad5af',
 'shopmodel:697585639fbc4c82b04b2f7b0cea500a',
 'shopcase:e1efa9a4c0584419a856c54d504f4fcd',
 'shopfile:8d5bff7e334e438dbd7e6a22d558225e',
 'shopfile:6c8bd87ba57841d6be9ac50a2c34add7',
 'shopcase:6454261bed644b159e01016064fe5790',
 'shopfile:0fefbc2bd6ef4105934e66a73e7b7b0c',
 'shopfile:3f786d18ac0b46da83f6081aa6e24772',
 'shop_result__7df1c42b-fc68-4895-b666-93d3cfb5a60b',
 'shop_result__cf536392-48a0-4e1e-910c-9bcbe7f848f9']

In [82]:
powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='shopscenario:c959c876201f435ca770bcf1a28ad47a'), NodeId(space='power_ops_instances', external_id='shopmodel:f337d96c4ad943c1a4232e2423bbfc6e'), NodeId(space='power_ops_instances', external_id='shopcase:452ba4f84d534ac38ad78356b3d10e93'), NodeId(space='power_ops_instances', external_id='shopfile:245b7a660c224c8da69d60f6280bad8c'), NodeId(space='power_ops_instances', external_id='shopfile:353ca5327e8d409988132e7c0f26639e'), NodeId(space='power_ops_instances', external_id='shop_result__d7329327-5c92-4c92-bafa-f94ac1aca550'), NodeId(space='power_ops_instances', external_id='shopscenario:4ed8eecfe7484e21b5baf1cc616f47a2'), NodeId(space='power_ops_instances', external_id='shopmodel:d37fce4661d44014a23f292c1098b1a8'), NodeId(space='power_ops_instances', external_id='shopscenario:eef230c5d6284153b88e29232b8ad5af'), NodeId(space='power_ops_instances', external_id='shopmodel:697585639fbc4c82b04b2f7b0cea500a'), NodeId(s